In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
from sklearn import preprocessing, svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

# for jupyter notebook
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# read in features

df = pd.read_csv('/home/sohaib/FEATURES.csv')
df = df.dropna(axis=0)

In [ ]:
df = df.drop(['HADM_ID'], axis=1)

# Categorical values

convert categorical values into one-hot-encoded vectors

In [ ]:
cat_value = ['DBSOURCE', 'FIRST_CAREUNIT' ,'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'INSURANCE', 'ICD9_CODE_x', 'GENDER', 'SEASON', 'ICD9_CODE_y']

In [ ]:
target = 'LOS'

In [ ]:
cont_values = ['HeartRate', 'sysBP', 'diasBP', 'oxygen_met', 'respRate', 'gluc', 'AGE']

In [ ]:
# give every category a number
# df['ADMISSION_TYPE'] = df['ADMISSION_TYPE'].astype('category').cat.codes

for cat in cat_value:
    # give each category a one-hot-encoded vector
    df = df.join(pd.get_dummies(df[cat], prefix=cat + "_"))

In [ ]:
df

In [ ]:
# remove columns
df = df.drop(cat_value, axis = 1)

df.set_index('ICUSTAY_ID', inplace=True)

In [ ]:
#df = df[df['LOS'] < 15]

df[df['AGE'] > 15]

In [ ]:
import seaborn as sns
print("Information about number of days in ICU \n")

# print median, max and min
median =  df['LOS'].median()
max_val =  df['LOS'].max()
min_val =  df['LOS'].min()
print("Median: ", median)
print("Max: ", max_val)
print("Min: ", min_val)

# plot diagram
x = df['LOS'].dropna(axis=0)
x = x.values

ax = sns.distplot(x)
ax

In [ ]:
x = df['diasBP']
y = df['LOS']


plt.scatter(x,y, label="AGE", color='b')

plt.xlabel('AGE')
plt.ylabel('LOS')

plt.title("Age distrubution")

# only needed if more plots are in one plt
plt.legend()
plt.show()

In [ ]:
def maplos(x):
    if x < 1.0: # short stay
        return 0
    if x < 2.5: # middel stay
        return 2
    if x < 200.0: # long stay
        return 3

df['LOS'] = df['LOS'].map(lambda x: maplos(x))

In [ ]:
df

# shuffle

In [ ]:
# shuffle

## pandas can shuffle to by index, but sentdex says thats ugly
df = sklearn.utils.shuffle(df)

df

In [ ]:
y = df[target].values
X = df.drop(target, axis=1).values

In [ ]:
#co = X[[cont for cont in cont_values]]
#co = preprocessing.scale(co)
#X = X.join(co)

#X = X.drop(cont_values, axis = 1).values
#X = np.concatenate((X, co), axis=1)

# scale
X = preprocessing.scale(X)
X

In [ ]:
X

In [ ]:
y=y.astype('int')

y

# Split data

In [ ]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [ ]:
test_size = 400

X_train = X[:-test_size]
y_train = y[:-test_size]

X_test = X[-test_size:]
y_test = y[-test_size:]

In [ ]:
X_train.shape

# NN

In [ ]:
#activation 
# - sigmoid = logistic
# - relu
# - tanh

#solver
# gradient decent - sgd

#hidden_layer_sieze
# - 45,90 take some time -- 16min
# - 10,15 lesser time -- 13min
# - 100, 150 -- 48min
# - 1000, 1500 time -- 2h 4min

nn=MLPClassifier(activation='logistic', solver='sgd', hidden_layer_sizes=(10,15), verbose=1)

In [ ]:
%%time
nn.fit(X_train, y_train)

In [ ]:
pred=nn.predict(X_test)

In [ ]:
count = 0

In [ ]:
for i in range(len(y_test)):
    if pred[i]==y_test[i]:
        count += 1

In [ ]:
x = df['diasBP']
y = df['LOS']


plt.scatter(x,y, label="AGE", color='b')

plt.xlabel('AGE')
plt.ylabel('LOS')

plt.title("Age distrubution")

# only needed if more plots are in one plt
plt.legend()
plt.show()

In [ ]:
count

In [ ]:
len(pred)

In [ ]:
count/len(pred)

## Other algorithmes

In [ ]:
clf = svm.SVR(kernel="linear")
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

# Neural Network

In [ ]:
# normalize??



In [ ]:
# split into train, valid, test

# features capital x, and lowercase y is labels